In [1]:
from doped import vasp_input
from doped.pycdt.core.defectsmaker import ChargedDefectsStructures
from pymatgen.core.structure import Structure

In [2]:
# Load our relaxed bulk cell (CdTe in conventional cubic structure), and make our defect supercell:
relaxed_conventional_cell = Structure.from_file("Examples/relaxed_conventional_POSCAR")

# make a 2x2x2 supercell expansion of our cubic conentional cell:
bulk_supercell = relaxed_conventional_cell.copy()
bulk_supercell.make_supercell(2)
bulk_supercell.to(fmt="poscar", filename="Bulk_Supercell_POSCAR") # calculate this with the same INCAR, KPOINTS and POTCAR settings as your defects!

In [3]:
def_structs = ChargedDefectsStructures(
    bulk_supercell,
    cellmax=bulk_supercell.num_sites,
    substitutions={"Te": ["Se"]},  # As a dictionary
    antisites_flag=True,
    include_interstitials=True,
)

Setting up vacancies
Setting up antisites
Searching for Voronoi interstitial sites (this can take a while)
Found the interstital sites, setting up interstitials

Number of jobs created:
    bulk = 1
    vacancies:
        vac_1_Cd = 5 with site multiplicity 32
        vac_2_Te = 5 with site multiplicity 32
    substitutions:
        sub_2_Se_on_Te = 7 with site multiplicity 32
        as_1_Cd_on_Te = 7 with site multiplicity 32
        as_1_Te_on_Cd = 7 with site multiplicity 32
    interstitials:
        inter_1_Cd = 3 with site multiplicity 32
        inter_2_Cd = 3 with site multiplicity 128
        inter_3_Cd = 3 with site multiplicity 1
        inter_4_Te = 9 with site multiplicity 1
        inter_5_Te = 9 with site multiplicity 128
        inter_6_Te = 9 with site multiplicity 32
Total (non dielectric) jobs created = 68



In [4]:
vacancies_only = {"vacancies": [val for val in def_structs.defects["vacancies"]]}
substitutions_only = {
    "substitutions": [val for val in def_structs.defects["substitutions"]]
}
interstitials_only = {
    "interstitials": [val for val in def_structs.defects["interstitials"]]
}

In [5]:
cdte_vacancies = vasp_input.prepare_vasp_defect_inputs(vacancies_only)
# Generates a dictionary of folders for VASP defect calculations, matching those written by ShakeNBreak

# Can do the same with the substitutions and interstitials with:
# cdte_substitutions = prepare_vasp_defect_inputs(substitutions_only)
# cdte_interstitials = prepare_vasp_defect_inputs(interstitials_only)

vac_1_Cd_-2
vac_1_Cd_-1
vac_1_Cd_0
vac_1_Cd_1
vac_1_Cd_2
vac_2_Te_-2
vac_2_Te_-1
vac_2_Te_0
vac_2_Te_1
vac_2_Te_2


In [6]:
for key, val in cdte_vacancies.items():
    vasp_input.vasp_std_files(
        val,
        input_dir=key,
        incar_settings={"ENCUT": 350},  # Custom INCAR settings; any flags that aren't numbers or True/False need to be input as strings with quotation marks
        #kpoints_settings={"generation_style": "Monkhorst", "kpoints": [[3, 3, 3]]}, # Custom KPOINTS settings
    )
# Prepare the vasp_std relaxation input files (in "{defect_folder}/vasp_std")
# Highly recommended to look at output INCARs or doped.vasp_input source code,
# to see what the default INCAR settings are.

In [7]:
from monty.serialization import loadfn, dumpfn

In [8]:
dumpfn(cdte_vacancies, "cdte_vacancies.json")

In [9]:
cdte_vacancies = loadfn("cdte_vacancies.json")

In [10]:
for key, val in cdte_vacancies.items():
    vasp_input.vasp_std_files(
        val,
        input_dir=key,
        incar_settings={"ENCUT": 350},  # Custom INCAR settings; any flags that aren't numbers or True/False need to be input as strings with quotation marks
        unperturbed_poscar=True,
        #kpoints_settings={"generation_style": "Monkhorst", "kpoints": [[3, 3, 3]]}, # Custom KPOINTS settings
    )
# Prepare the vasp_std relaxation input files (in "{defect_folder}/vasp_std")
# Highly recommended to look at output INCARs or doped.vasp_input source code,
# to see what the default INCAR settings are.

In [11]:
for key, val in cdte_vacancies.items():
    vasp_input.vasp_ncl_files(
        val,
        input_dir=key,
        incar_settings={"ENCUT": 350},  # Custom INCAR settings; any flags that aren't numbers or True/False need to be input as strings with quotation marks
        unperturbed_poscar=True,
        #kpoints_settings={"generation_style": "Monkhorst", "kpoints": [[3, 3, 3]]}, # Custom KPOINTS settings
    )
# Prepare the vasp_std relaxation input files (in "{defect_folder}/vasp_std")
# Highly recommended to look at output INCARs or doped.vasp_input source code,
# to see what the default INCAR settings are.

All runs as expected